In [1]:
import pandas as pd
import numpy as np

#### 수작업으로 변수 선택. 명세서가 부실한 변수와 NA가 많은 변수는 제거

In [3]:
accident_data = []
person_data = []
vehicle_data = []
for q in [1,2,3,4]:
    accident_data.append(pd.read_excel('data/Crash_Qtr0'+str(q)+'_2015.xlsx', sheetname = 0))
    person_data.append(pd.read_excel('data/Crash_Qtr0'+str(q)+'_2015.xlsx', sheetname = 1))
    vehicle_data.append(pd.read_excel('data/Crash_Qtr0'+str(q)+'_2015.xlsx', sheetname = 2))

In [4]:
accidents = pd.concat(accident_data).reset_index(drop=True)
persons = pd.concat(person_data).reset_index(drop=True)
vehicles = pd.concat(vehicle_data).reset_index(drop=True)

In [7]:
accidents.dtypes

ACC_DATE                object
ACC_TIME                object
AGENCY_CODE             object
AREA_CODE               object
COLLISION_TYPE_CODE      int64
COUNTY_NO              float64
C_M_ZONE_FLAG           object
DISTANCE               float64
DISTANCE_DIR_FLAG       object
FEET_MILES_FLAG         object
FIX_OBJ_CODE           float64
HARM_EVENT_CODE1       float64
HARM_EVENT_CODE2       float64
JUNCTION_CODE          float64
LANE_CODE              float64
LATITUDE               float64
LIGHT_CODE             float64
LOC_CODE                object
LOGMILE_DIR_FLAG        object
LOG_MILE               float64
LONGITUDE              float64
MAINROAD_NAME           object
MUNI_CODE              float64
RD_COND_CODE           float64
RD_DIV_CODE            float64
REFERENCE_NO           float64
REFERENCE_ROAD_NAME     object
REFERENCE_SUFFIX        object
REFERENCE_TYPE_CODE     object
REPORT_NO               object
ROUTE_TYPE_CODE         object
RTE_NO                 float64
RTE_SUFF

In [8]:
persons.dtypes

ALCOHOL_TESTTYPE_CODE    float64
ALCOHOL_TEST_CODE        float64
BAC_CODE                 float64
CDL_FLAG                  object
CLASS                     object
CONDITION_CODE           float64
DATE_OF_BIRTH             object
DRUG_TESTRESULT_CODE      object
DRUG_TEST_CODE           float64
EJECT_CODE               float64
EMS_UNIT_LABEL            object
EQUIP_PROB_CODE          float64
FAULT_FLAG                object
INJ_SEVER_CODE             int64
LICENSE_STATE_CODE        object
MOVEMENT_CODE            float64
OCC_SEAT_POS_CODE        float64
PED_LOCATION_CODE        float64
PED_OBEY_CODE            float64
PED_TYPE_CODE            float64
PED_VISIBLE_CODE         float64
PERSON_ID                 object
PERSON_TYPE               object
REPORT_NO                 object
SAF_EQUIP_CODE           float64
SEX_CODE                  object
VEHICLE_ID                object
dtype: object

In [9]:
vehicles.dtypes

AREA_DAMAGED_CODE1           float64
AREA_DAMAGED_CODE2           float64
AREA_DAMAGED_CODE3           float64
AREA_DAMAGED_CODE_IMP1       float64
AREA_DAMAGED_CODE_MAIN       float64
BODY_TYPE_CODE               float64
COMMERCIAL_FLAG               object
CONTI_DIRECTION_CODE          object
CV_BODY_TYPE_CODE            float64
DAMAGE_CODE                    int64
DRIVERLESS_FLAG               object
FIRE_FLAG                     object
GOING_DIRECTION_CODE          object
GVW_CODE                     float64
HARM_EVENT_CODE              float64
HAZMAT_SPILL_FLAG             object
HIT_AND_RUN_FLAG              object
HZM_NUM                       object
MOVEMENT_CODE                float64
NUM_AXLES                    float64
PARKED_FLAG                   object
REPORT_NO                     object
SPEED_LIMIT                  float64
TOWED_AWAY_FLAG               object
TOWED_VEHICLE_CONFIG_CODE    float64
VEHICLE_ID                    object
VEH_MAKE                      object
V

## 불필요한 변수 제거

In [4]:
merged_data = pd.merge(persons, accidents, how='inner',on='REPORT_NO')
merged_data = pd.merge(merged_data, vehicles, how='left',on="VEHICLE_ID")
merged_data.drop('REPORT_NO_y', axis=1, inplace=True)
merged_data.rename(columns = {'REPORT_NO_x' : 'REPORT_NO'}, inplace = True)

In [5]:
merged_data.drop('HARM_EVENT_CODE', axis=1, inplace=True)
merged_data.drop('HARM_EVENT_CODE1', axis=1, inplace=True)
merged_data.drop('HARM_EVENT_CODE2', axis=1, inplace=True)
merged_data.drop('FIX_OBJ_CODE', axis=1, inplace=True)
merged_data.drop('DRUG_TEST_CODE', axis=1, inplace=True)
merged_data.drop('ALCOHOL_TEST_CODE', axis=1, inplace=True)
merged_data.drop('VEH_MAKE', axis=1, inplace=True)
merged_data.drop('EJECT_CODE', axis=1, inplace=True)

data.drop('COMMERCIAL_FLAG', axis=1, inplace=True)
data.drop('SIGNAL_FLAG', axis=1, inplace=True)

## 사고 발생 일시 관련 변수 통합

In [6]:
merged_data['TIME'] = pd.to_datetime(merged_data.ACC_DATE + ' ' + merged_data.ACC_TIME)
merged_data.drop('ACC_DATE', axis=1, inplace=True)
merged_data.drop('ACC_TIME', axis=1, inplace=True)

## 나이 변수 생성. (Noise 제거를 위해 16세 이상, 90세 이하만 분석)

In [7]:
merged_data['AGE'] = merged_data.DATE_OF_BIRTH.apply(lambda x : 100 - int(str(x)[-2:]) + 15 if type(x) == str else x)
merged_data = merged_data.loc[np.array(merged_data.AGE >= 16) & np.array(merged_data.AGE <= 90)]
merged_data.drop('DATE_OF_BIRTH', axis=1, inplace=True)

In [8]:
merged_data.shape

(200118, 29)

In [10]:
merged_data.to_csv('data2distribute.csv', index=False)

# 변수값 코드북 참조해서 바꾸는거 여기서

## Step1_2의 내용